In [ ]:
spark.sql(
    """
CREATE OR REPLACE VIEW edl_views.view_30_days_nmis 
TBLPROPERTIES (
    "edl_sources" = "com.deere.enterprise.datalake.enhance.nmis_device_events_enhance"
) AS
SELECT
    date_trunc('DAY', e.event_timestamp) AS event_date,
    e.device_name,
    e.device_type,
    e.serial_number,
    CASE
        WHEN s.type = 'Proactive Memory Free' THEN 'Proactive Memory Used'
        ELSE s.type
    END AS type,
    CASE
        WHEN s.type = 'Proactive Memory Free' THEN 100 - s.value
        ELSE s.value
    END AS value
FROM
    edl_current.nmis_device_events e
JOIN
    edl.nmis_device_event_statuses s USING (event_id)
WHERE
    e.event_timestamp >= date_sub(current_date, 30)
    AND e.event_timestamp < CURRENT_DATE;
"""
)

In [ ]:
spark.sql(
    """
CREATE OR REPLACE VIEW edl_views.network_device_daily_metrics 
TBLPROPERTIES (
    "edl_sources" = "com.deere.enterprise.datalake.enhance.nmis_device_events_enhance"
) AS
SELECT
    device_name,
    serial_number,
    event_date,
    AVG(value) FILTER (WHERE type = 'Proactive CPU' AND device_type = 'switch') AS switch_cpu_avg,
    AVG(value) FILTER (WHERE type = 'Proactive Memory Used' AND device_type = 'switch') AS switch_mem_avg,
    AVG(value) FILTER (WHERE type = 'Proactive Interface Input Utilisation' AND device_type = 'switch') AS switch_interface_in_util_avg,
    AVG(value) FILTER (WHERE type = 'Proactive Interface Output Utilisation' AND device_type = 'switch') AS switch_interface_out_util_avg,
    AVG(value) FILTER (WHERE type = 'Proactive CPU' AND device_type = 'router') AS router_cpu_avg,
    AVG(value) FILTER (WHERE type = 'Proactive Memory Used' AND device_type = 'router') AS router_mem_avg,
    AVG(value) FILTER (WHERE type = 'Proactive Interface Input Utilisation' AND device_type = 'router') AS router_interface_in_util_avg,
    AVG(value) FILTER (WHERE type = 'Proactive Interface Output Utilisation' AND device_type = 'router') AS router_interface_out_util_avg
FROM
    edl_views.view_30_days_nmis
GROUP BY
    device_name, serial_number, event_date;
"""
)

In [ ]:
spark.sql(
    """
CREATE OR REPLACE VIEW edl_views.network_device_vits
TBLPROPERTIES (
    "edl_sources" = "com.deere.enterprise.datalake.enhance.nmis_device_events_enhance"    
) AS (
WITH distinct_network_devices AS (
    SELECT distinct device_name
    FROM edl_current.nmis_device_events
)
SELECT
    devices.device_name,
    COUNT(vits.risk_rating) AS risk_count,
    SUM(COALESCE(vits.risk_score, 0)) AS risk_score
FROM
    distinct_network_devices devices
LEFT JOIN
    edl_current.sn_vul_vulnerable_item vits ON LOWER(vits.cmdb_ci) = LOWER(devices.device_name)
WHERE
    vits.state NOT IN ('Closed', 'Resolved')
GROUP BY
    devices.device_name
)
"""
)

In [ ]:
spark.sql(
    """
CREATE OR REPLACE VIEW edl_views.network_device_snow_info
TBLPROPERTIES(
    "edl_sources" = "com.deere.enterprise.datalake.enhance.nmis_device_events_enhance"
)  AS (
    WITH distinct_network_devices AS (
    SELECT distinct device_name, serial_number
    FROM edl_current.nmis_device_events
),
    first_ci_match AS (
    SELECT nmis.device_name, nmis.serial_number, min(ci.sys_id) first_id
    FROM distinct_network_devices nmis
    JOIN edl_current.cmdb_ci ci
    ON lower(ci.name) = lower(nmis.device_name)
    AND lower(ci.serial_number) = lower(nmis.serial_number)
    AND ci.subcategory in ('Router', 'Switch')
    group by nmis.device_name, nmis.serial_number
),
cmdb_ci AS (
    SELECT first.device_name, first.serial_number, ci.assignment_group, ci.location
    FROM first_ci_match first
    JOIN edl_current.cmdb_ci ci
    ON first.first_id = ci.sys_id
),
first_asset_match AS (
    SELECT nmis.device_name, nmis.serial_number, min(alm.sys_id) first_id
    FROM distinct_network_devices nmis
    JOIN edl_current.alm_asset alm
    ON nmis.serial_number = alm.serial_number
    group by nmis.device_name, nmis.serial_number
),
alm_asset AS (
    SELECT first.device_name,
        first.serial_number,
        CAST(REGEXP_REPLACE(alm.cost, '[$,]', '') AS DECIMAL(15,2)) cost,
        alm.u_replenishment_date
    FROM first_asset_match first
    JOIN edl_current.alm_asset alm
    ON alm.sys_id = first.first_id
)
SELECT devices.device_name, 
    devices.serial_number, 
    ci.assignment_group, 
    ci.location, 
    alm.cost, 
    alm.u_replenishment_date
FROM distinct_network_devices devices
LEFT JOIN cmdb_ci ci
ON devices.device_name = ci.device_name
AND devices.serial_number = ci.serial_number
LEFT JOIN alm_asset alm
ON devices.device_name = alm.device_name
AND devices.serial_number = alm.serial_number   
)
"""
)